In [68]:
#!pip install torch
import sqlite3
import torch
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence
import pandas as pd

In [69]:
# Создаем соединение с базой данных
conn = sqlite3.connect('garpix_db_0.0.2.db')

In [70]:
# Формируем DataFrame  
df = pd.read_sql_query("""
    SELECT
        dr.calculation_id,
		b.size_width, 
		b.size_height, 
		b.size_length,
		dr.density_percent,
		dr.filling_space_percent
    FROM data_result dr
    JOIN boxes b ON b.calculation_id = dr.calculation_id
""", conn)
print(df.head())

# Группируем данные по calculate_id 
# Группируем DataFrame по calculate_id и создаем новый столбец с именем "width_list", 
# который содержит список всех значений size_width (ширина ящика) для каждой группы. 
grouped_df = df.groupby('calculation_id')['size_width'].apply(list).reset_index(name='width_list')
# Наконец, создаем список тензоров из этих списков значений size_width.
data = [torch.tensor(widths).unsqueeze(1) for widths in grouped_df['width_list']]

# Формируем список значений процента заполнения грузового пространства
# Группируем по calculation_id и извлекаем значения filling_space_percent 
grouped = df.groupby('calculation_id')['filling_space_percent'].unique()

# Создаем np.array уникальных значений filling_space_percent для каждого calculation_id
result = [[x] for x in grouped.values]
rand_tmp = np.array(result)

   calculation_id  size_width  size_height  size_length  density_percent  \
0           16900        98.0        143.0        385.0         62.12352   
1           16900       196.0        198.0        363.0         62.12352   
2           16900       277.0        190.0        407.0         62.12352   
3           16900       170.0        325.0        255.0         62.12352   
4           16900       250.0        350.0        160.0         62.12352   

   filling_space_percent  
0               61.22618  
1               61.22618  
2               61.22618  
3               61.22618  
4               61.22618  


In [71]:
import torch
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence

# Определяется модель Net - это нейронная сеть, 
# состоящая из слоя RNN (рекуррентная нейронная сеть) и полносвязного слоя. 
# Скрытый размерность RNN равен 16, а входной размерность равна 1.
# Модель использует функцию активации по умолчанию.
class Net(nn.Module):
# В конструкторе класса мы определяем наши слои: nn.RNN и nn.Linear. 
# Эти слои будут использоваться при проходе данных через модель. 
# Здесь мы указываем, что на входе имеем 1 признак, 
# размер скрытого состояния RNN равен 16, 
# а архитектура сети предполагает использование batch_first=True, 
# то есть размерность первой оси является размером пакета.
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=16, batch_first=True)
        self.fc = nn.Linear(16, 1)

# Функция forward принимает входной тензор x, 
# пропускает его через слой RNN, 
# получает выходной тензор и пропускает его через полносвязный слой, 
# чтобы получить выходной тензор.
    def forward(self, x):
        output, _ = self.rnn(x)
        x = self.fc(output[:, -1, :])
        return x.squeeze()  # удаляем избыточные размерности

model = Net()

# определяется устройство, на котором будет проходить обучение модели (GPU или CPU).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Определение функции потерь и оптимизатора
# Определяется функция потерь (criterion) - это среднеквадратичная ошибка (MSE).
criterion = nn.MSELoss()
# Определяется оптимизатор (optimizer) - 
# это алгоритм Adam с коэффициентом обучения 0.001, 
# который будет использоваться для обновления параметров модели.
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Генерация данных для обучения
# TODO тут вместо даты, нужно подставить наши значения (DONE)
import numpy as np
#num_samples = 1000
#data = []
#for i in range(num_samples):
#    input_size = np.random.randint(3, 10)
#    data.append(torch.from_numpy(np.random.rand(input_size, 1) * 100).float().to(device))
# Генерируется синтетический набор данных для обучения - это список data, 
# который состоит из 1000 тензоров, 
# каждый из которых имеет случайный размер от 3 до 10 и 
# содержит случайные числа от 0 до 100. 
# Каждый тензор data заполняется нулями до максимальной длины, 
# чтобы все тензоры имели одинаковую длину. 
# Кроме того, создается тензор меток labels, 
# который состоит из 1000 случайных чисел от 0 до 100.
data_tensor = pad_sequence(data, batch_first=True)  # заполняем более короткие тензоры нулями
# np.random.rand(num_samples, 1) создает массив размером (num_samples, 1) 
# со случайными значениями в диапазоне от 0 до 1.
# .to(device) перемещает тензор на устройство, указанное в переменной device, если оно доступно.
# TODO сюда вместо rand_tmp вставляем значени, что хотим получить  (DONE)
#rand_tmp = np.random.rand(num_samples, 1) * 100
# перемещение labels на устройство обусловлено тем, 
# что он используется в вычислении функции потерь, 
# которая определена на устройстве.
labels = torch.from_numpy(rand_tmp).float().to(device)
# изменяем размерность целевого тензора
# Аргумент -1 в данном случае означает, 
# что размерность должна быть вычислена автоматически 
# на основе других размерностей тензора.
labels = labels.view(-1)


# обучение модели - модель пропускает каждый тензор из data через себя, 
# получая предсказание output, 
# затем сравнивает его с соответствующей меткой target, 
# используя функцию потерь criterion, и вычисляет ошибку loss. 
# Далее выполняется обратное распространение ошибки и обновление весов optimizer.step(). 
# Процесс обучения повторяется в цикле 100 раз.
for epoch in range(100):
# Обнуление градиента всех параметров модели. 
# Это необходимо делать перед каждым проходом (эпохой) обучения модели, 
# чтобы избежать накопления градиентов от предыдущих проходов. 
# Без этого может произойти неожиданное обновление весов, 
# которое может ухудшить качество модели.
    optimizer.zero_grad()

# Вызываем метод forward модели model с тензором data_tensor в качестве входных данных
    output = model(data_tensor)
    target = labels

# Вычисляем функцию потерь между предсказаниями модели output[-1] и истинными значениями target.
    loss = criterion(output[-1], target)

# Выполняем обратное распространение ошибки и обновляем веса
	# выполняет обратное распространение ошибки (backpropagation) в нейронной сети. 
	# Она вычисляет градиенты функции потерь по всем параметрам модели
    loss.backward()
	# обновление весов модели с помощью оптимизатора 
    optimizer.step()

# loss.item() — значению функции потерь на данной эпохе. 
# Функция потерь (MSE) измеряет разницу между прогнозируемым значением и 
# фактическим значением, поэтому чем меньше значение функции потерь, 
# тем лучше модель обучена на данном этапе
    if epoch % 10 == 0:
        print('Epoch {}, Loss {}'.format(epoch, loss.item()))

/Users/graur/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([628])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Loss 3343.9140625
Epoch 10, Loss 3330.6708984375
Epoch 20, Loss 3315.8056640625
Epoch 30, Loss 3297.2392578125
Epoch 40, Loss 3270.699462890625
Epoch 50, Loss 3227.579833984375
Epoch 60, Loss 3167.64208984375
Epoch 70, Loss 3113.173583984375
Epoch 80, Loss 3070.112548828125
Epoch 90, Loss 3034.38427734375


In [72]:
conn.close()